In [1]:
from huggingface_hub import InferenceClient
import time

c:\Users\Danjin\miniconda3\envs\pdftrans\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

# Load environment variables from .env file
load_dotenv()

# Get Hugging Face token from environment
HF_TOKEN = os.getenv("HF_TOKEN")

# Initialize client with Llama 3.1 model
client = InferenceClient(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    token=HF_TOKEN
)

def run_resume_rewriter(prompt: str) -> str:
    """Send prompt to model and get rewritten bullet point"""
    try:
        messages = [{"role": "user", "content": prompt}]
        
        response = client.chat_completion(
            messages=messages,
            max_tokens=150,
            temperature=0.7
        )
        
        if response and response.choices:
            result = response.choices[0].message.content.strip()
            
            # Clean up response if it includes extra formatting
            if "rewritten bullet point:" in result.lower():
                return result.split(":")[-1].strip()
            elif "rewritten:" in result.lower():
                return result.split(":")[-1].strip()
            
            return result
        
        return "[Error: No response received]"
    
    except Exception as e:
        return f"[Error: {str(e)}]"

print("Hugging Face InferenceClient setup complete!")

# Test the connection
test_prompt = "Rewrite this bullet point to be stronger: • Managed emails for team."
test_result = run_resume_rewriter(test_prompt)
print(f"Test result: {test_result}")


Hugging Face InferenceClient setup complete!
Test result: • Effectively managed and responded to a high volume of emails for the team, ensuring timely and accurate communication and minimizing email-related delays or issues.


In [3]:
# CELL 3: Load Files
def load_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read().strip()

# Load your files
prompt_template = load_file("prompt/direct_instruction.txt")
bullet_content = load_file("bullets.txt")

# Split bullet points
bullet_points = [line.strip() for line in bullet_content.split('\n') if line.strip().startswith('•')]

print(f"Loaded {len(bullet_points)} bullet points:")
for i, bullet in enumerate(bullet_points, 1):
    print(f"{i}. {bullet}")

Loaded 6 bullet points:
1. • Scheduled appointments, managed emails and calls for senior leadership.
2. • Provided end-to-end event logistics and meeting support.
3. • Familiarity with audit techniques, processes, and tools.
4. • Proficiency in using GST portals for registration, payment, and return filing.
5. • Extracted, cleaned & integrated multi-source datasets (Insurance Records, Customer Feedback); performed data profiling & executed sentiment analysis via Power Query to deliver customer-centric insights.
6. • Designed & deployed dynamic dashboards using Bar, Line, Ribbon, Donut & Matrix visualizations; implemented Role-Based Security (RLS) & scheduled automated data refresh for real-time, secure reporting.


In [5]:
# CELL 4: Process Bullet Points
all_results = []

for i, bullet in enumerate(bullet_points, 1):
    print(f"\nProcessing bullet {i}/{len(bullet_points)}...")
    
    # Replace placeholder with actual bullet
    full_prompt = prompt_template.replace("{{bullet}}", bullet.strip())
    
    print(f"Original: {bullet}")
    
    # Process with model
    try:
        generated_text = run_resume_rewriter(full_prompt)
        
        print(f"Rewritten: {generated_text}")
        
        all_results.append({
            'original': bullet,
            'rewritten': generated_text
        })
        
        # Small delay to avoid rate limiting
        time.sleep(2)
        
    except Exception as e:
        print(f"Error processing bullet {i}: {e}")
        all_results.append({
            'original': bullet,
            'rewritten': f"Error: {e}"
        })
    
    print("-" * 50)


Processing bullet 1/6...
Original: • Scheduled appointments, managed emails and calls for senior leadership.
Rewritten: - Effectively coordinated senior leadership schedules, streamlined email and call management processes.
--------------------------------------------------

Processing bullet 2/6...
Original: • Provided end-to-end event logistics and meeting support.
Rewritten: - Coordinated comprehensive event logistics, meeting planning, and execution with attention to detail and precision.
--------------------------------------------------

Processing bullet 3/6...
Original: • Familiarity with audit techniques, processes, and tools.
Rewritten: Applied expert knowledge of audit methodologies, processes, and tools to drive efficient risk assessment and compliance evaluations.
--------------------------------------------------

Processing bullet 4/6...
Original: • Proficiency in using GST portals for registration, payment, and return filing.
Rewritten: Effectively managed GST complian